In [24]:
from psaw import PushshiftAPI
import datetime as dt
import json
import requests 
import networkx as nx
import time
import datetime

In [25]:
# Global variables
api = PushshiftAPI()
comment_ids_endpoint_url = 'https://api.pushshift.io/reddit/submission/comment_ids/'
comment_endpoint_url = 'https://api.pushshift.io/reddit/comment/search/'
submission_endpoint_url = 'https://api.pushshift.io/reddit/submission/search/'
start_epoch=int(dt.datetime(2018, 6, 30).timestamp())
end_epoch=int(dt.datetime(2018, 7, 2).timestamp())

In [34]:
relevant_subreddits = [
    'best_of_crypto',
    'bitcoin',
    'bitcoinbeginners',
    'bitcoincash',
    'bitcoindiscussion',
    'bitcoinmarkets',
    'bitcoinmining',
    'bitcointechnology',
    'bitcointrading',
    'btc',
    'buttcoin',
    'cryptocurrency',
    'cryptocurrencytesting',
    'cryptomarkets',
    'cryptopia',
    'cryptopump',
    'cryptotrade',
    'ethanalysis',
    'ethdapps',
    'ethdev',
    'ethereum',
    'ethereumcommunity',
    'ethereumnoobies',
    'ethermining',
    'ethinsider',
    'ethinvestor',
    'ethtrader',
    'ethtraderpro',
    'gpumining',
    'ripple'
]

In [41]:
keyword = 't.me'

In [42]:
start_time = time.time()
print("Starting at " + str(datetime.datetime.now()))

subreddits_set = set()

submission_url = submission_endpoint_url + "?q=" + keyword \
            + "&after=" + str(start_epoch) + "&before=" + str(end_epoch) \
            + "&fields=id,author,selftext,title,num_comments,created_utc," \
            + "score,full_link,url,subreddit,subreddit_id"

sub_counter = 0

with open('pum_n_dump_data_from_api/' + keyword + "_reddit_submissions" + "_" \
          + str(start_epoch) + "_" + str(end_epoch) + ".txt", "w") as s_f, \
open('pum_n_dump_data_from_api/' + keyword + "_reddit_comments" + "_from_submissions_"\
     + str(start_epoch) + "_" + str(end_epoch) + ".txt", "w") as c_f:
    
    sub_r = requests.get(url = submission_url)
    print("Submission URL:", submission_url)
    
    if sub_r.status_code == 200:
        sub_json_obj = json.loads(sub_r.text)
        sub_arr = sub_json_obj['data']

    for s in sub_arr:

        subreddit = s['subreddit']
        subreddits_set.add(subreddit)
        
        if subreddit.lower() in relevant_subreddits:
            sub_id = s['id']
            s_f.write(str(s) + '\n')

            sub_counter += 1
            if sub_counter % 1000 == 0:
                print("Adding ", sub_counter, "th submission")

            comment_ids_url = comment_ids_endpoint_url + str(sub_id)
            r = requests.get(url = comment_ids_url)

            if r.status_code == 200:
                comment_ids_json_obj = json.loads(r.text)
                comment_ids_arr = comment_ids_json_obj['data']

                if len(comment_ids_arr) > 0 :                
                    comment_ids_str = ','.join(comment_ids_arr)

                    comment_url = comment_endpoint_url + "?ids=" + comment_ids_str \
                    + "&size=" + str(10) \
                    + "&fields=id,author,body,created_utc,score," \
                    + "parent_id,link_id,subreddit,subreddit_id"

                    com_r = requests.get(url = comment_url)
                    if com_r.status_code == 200:
                        comment_json_obj = json.loads(com_r.text)
                        comment_ids_arr = comment_json_obj['data']

                        c_counter = 0
                        for c in comment_ids_arr:
                            c_f.write(str(c) + '\n')
                            c_counter += 1

                            if c_counter % 1000 == 0:
                                print("Adding ",c_counter, "th comment to ", sub_counter, "th submission")            


print("Finished at " + str(datetime.datetime.now()))
print("Total time: --- {0} seconds ---".format(round(time.time() - start_time, 4)))

Starting at 2019-10-31 17:53:35.827629
Submission URL: https://api.pushshift.io/reddit/submission/search/?q=t.me&after=1530331200&before=1530504000&fields=id,author,selftext,title,num_comments,created_utc,score,full_link,url,subreddit,subreddit_id
Finished at 2019-10-31 17:53:39.351638
Total time: --- 3.5246 seconds ---


In [43]:
start_time = time.time()
print("Starting at " + str(datetime.datetime.now()))

subreddits_set_2 = set()

comment_url = comment_endpoint_url + "?q=" + keyword \
            + "&after=" + str(start_epoch) + "&before=" + str(end_epoch) \
            + "&fields=id,author,body,created_utc,score," \
            + "parent_id,link_id,subreddit,subreddit_id"

c_counter = 0

with open('pum_n_dump_data_from_api/' + keyword + "_reddit_comments" + "_"\
     + str(start_epoch) + "_" + str(end_epoch) + ".txt", "w") as c_f:
    
    c_r = requests.get(url = comment_url)
    print("Comment URL:", comment_url)
    
    if c_r.status_code == 200:
        c_json_obj = json.loads(c_r.text)
        c_arr = c_json_obj['data']

    for c in c_arr:

        subreddit = c['subreddit']
        subreddits_set_2.add(subreddit)
        
        if subreddit.lower() in relevant_subreddits:
            c_id = c['id']
            c_f.write(str(c) + '\n')

            c_counter += 1
            if c_counter % 1000 == 0:
                print("Adding ", c_counter, "th submission")

print("Finished at " + str(datetime.datetime.now()))
print("Total time: --- {0} seconds ---".format(round(time.time() - start_time, 4)))

Starting at 2019-10-31 17:53:39.366766
Comment URL: https://api.pushshift.io/reddit/comment/search/?q=t.me&after=1530331200&before=1530504000&fields=id,author,body,created_utc,score,parent_id,link_id,subreddit,subreddit_id
Finished at 2019-10-31 17:53:39.795931
Total time: --- 0.4293 seconds ---
